# Práctica Neo4j
- Martínez Ostoa Néstor Iván 
- Bases de Datos No Estructuradas
- LCD 
- IIMAS, UNAM

In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
products_df = pd.read_csv('data/visitas_profeco.csv')
print(products_df.shape)
print(products_df.columns)
products_df.head()

(31214, 13)
Index(['ANIO', 'UNIDAD_ADMINISTRATIVA', 'NUMERO_EXPEDIENTE', 'FECHA_VISITA',
       'NOM_VERIFICADA', 'TIPO_PRODUCTO', 'DESCRIPCION_PRODUCTO',
       'RAZON_SOCIAL', 'GIRO_COMERCIAL', 'MUNICIPIO', 'ENTIDAD',
       'INCUMPLIMIENTO', 'RESULTADO'],
      dtype='object')


,ANIO,UNIDAD_ADMINISTRATIVA,NUMERO_EXPEDIENTE,FECHA_VISITA,NOM_VERIFICADA,TIPO_PRODUCTO,DESCRIPCION_PRODUCTO,RAZON_SOCIAL,GIRO_COMERCIAL,MUNICIPIO,ENTIDAD,INCUMPLIMIENTO,RESULTADO
0,2015,AGUASCALIENTES,702/000009/2015,2015-01-05,NOM-015-SCFI-2007,MU�ECOS,EN EMPAQUE DE CARTON ESTAMPADO Y PLASTICO RIGI...,JULIO CEPEDA JUGUETERIAS S.A. DE C.V.,JUGUETERIA,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
1,2015,AGUASCALIENTES,702/000012/2015,2015-01-06,NOM-004-SCFI-2006,ROPA CABALLERO,CAMISA PARA CABALLERO EN COLOR AZUL MARINO DE ...,C&A DE MEXICO S. DE R.L.,TIENDA DEPARTAMENTAL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
2,2015,AGUASCALIENTES,702/000132/2015,2015-02-12,NOM-142-SSA1-1995,BRANDY,UNA BOTELLA BRANDY DE LA MARCA TORRES 10 CON L...,ASFEIXAS S.A. DE C.V.,MOTEL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
3,2015,AGUASCALIENTES,702/000133/2015,2015-02-12,NOM-142-SSA1-1995,BRANDY,ENVASE DE VIDRIO CONTENIDO BRANDY DE LA MARCA ...,MOTEL VILLA MIRO- MIRO HOTELERIA S.A. DE C.V.,MOTEL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA
4,2015,AGUASCALIENTES,702/000134/2015,2015-02-12,NOM-142-SSA1-1995,BRANDY,BOTELLA DE VIDRIO CONTENIDO BRANDY 100% UVA GR...,MOTEL LUA- COMPA�IA HOTELERA PESCADORES S.A. ...,MOTEL,AGUASCALIENTES,AGUASCALIENTES,NO SE DETECTO INCUMPLIMIENTO,VISITA CORRECTA


In [5]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31214 entries, 0 to 31213
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ANIO                   31214 non-null  int64 
 1   UNIDAD_ADMINISTRATIVA  31214 non-null  object
 2   NUMERO_EXPEDIENTE      31214 non-null  object
 3   FECHA_VISITA           31214 non-null  object
 4   NOM_VERIFICADA         31214 non-null  object
 5   TIPO_PRODUCTO          31214 non-null  object
 6   DESCRIPCION_PRODUCTO   31214 non-null  object
 7   RAZON_SOCIAL           31214 non-null  object
 8   GIRO_COMERCIAL         31214 non-null  object
 9   MUNICIPIO              31214 non-null  object
 10  ENTIDAD                31214 non-null  object
 11  INCUMPLIMIENTO         31214 non-null  object
 12  RESULTADO              31214 non-null  object
dtypes: int64(1), object(12)
memory usage: 3.1+ MB


In [6]:
def analyze_column(col, all=False):
    l = products_df[col].unique()
    print(f"Shape: {l.shape}")
    print("--------------")
    if l.shape[0] <= 100:
        print(l)
    else:
        if all:
            print(l)
        else:
            print(l[:50])

In [7]:
analyze_column('UNIDAD_ADMINISTRATIVA')

Shape: (49,)
--------------
['AGUASCALIENTES' 'BAJA CALIFORNIA' 'BAJA CALIFORNIA SUR' 'CAMPECHE'
 'CENTRO' 'CHIAPAS' 'CHIHUAHUA' 'CHIHUAHUA SUBDELEGACION' 'CIUDAD OBREGON'
 'COAHUILA' 'COLIMA' 'DIRECCION GENERAL DE VERIFICACION Y VIGILANCIA'
 'DURANGO' 'ECATEPEC' 'GUANAJUATO' 'GUERRERO' 'HIDALGO' 'IGUALA' 'JALISCO'
 'MANZANILLO' 'MEXICALI' 'MICHOACAN' 'MORELOS' 'NAUCALPAN' 'NAYARIT'
 'NEZAHUALCOYOTL' 'NUEVO LEON' 'OAXACA' 'ORIENTE' 'PLAYA DEL CARMEN'
 'PONIENTE' 'PUEBLA' 'QUERETARO' 'QUINTANA ROO' 'REYNOSA'
 'SAN LUIS POTOSI' 'SINALOA' 'SONORA' 'SUR' 'TABASCO' 'TAMAULIPAS'
 'TAPACHULA' 'TLALNEPANTLA' 'TLAXCALA' 'TOLUCA' 'TORREON' 'VERACRUZ'
 'YUCATAN' 'ZACATECAS']


## Preprocesamiento
- Conversión a catálogos
    - Estados
    - Tiendas
    - Productos
    - Relación Productos Tiendas

In [8]:
def clean_string(series):
    s = series.astype(str)
    s = s.str.strip()
    s = s.str.lower()
    s = s.replace(' +', ' ', regex=True)
    return s

In [9]:
products_df = products_df.apply(clean_string, axis=1)
products_df.head()

,ANIO,UNIDAD_ADMINISTRATIVA,NUMERO_EXPEDIENTE,FECHA_VISITA,NOM_VERIFICADA,TIPO_PRODUCTO,DESCRIPCION_PRODUCTO,RAZON_SOCIAL,GIRO_COMERCIAL,MUNICIPIO,ENTIDAD,INCUMPLIMIENTO,RESULTADO
0,2015,aguascalientes,702/000009/2015,2015-01-05,nom-015-scfi-2007,mu�ecos,en empaque de carton estampado y plastico rigi...,julio cepeda jugueterias s.a. de c.v.,jugueteria,aguascalientes,aguascalientes,no se detecto incumplimiento,visita correcta
1,2015,aguascalientes,702/000012/2015,2015-01-06,nom-004-scfi-2006,ropa caballero,camisa para caballero en color azul marino de ...,c&a de mexico s. de r.l.,tienda departamental,aguascalientes,aguascalientes,no se detecto incumplimiento,visita correcta
2,2015,aguascalientes,702/000132/2015,2015-02-12,nom-142-ssa1-1995,brandy,una botella brandy de la marca torres 10 con l...,asfeixas s.a. de c.v.,motel,aguascalientes,aguascalientes,no se detecto incumplimiento,visita correcta
3,2015,aguascalientes,702/000133/2015,2015-02-12,nom-142-ssa1-1995,brandy,envase de vidrio contenido brandy de la marca ...,motel villa miro- miro hoteleria s.a. de c.v.,motel,aguascalientes,aguascalientes,no se detecto incumplimiento,visita correcta
4,2015,aguascalientes,702/000134/2015,2015-02-12,nom-142-ssa1-1995,brandy,botella de vidrio contenido brandy 100% uva gr...,motel lua- compa�ia hotelera pescadores s.a. d...,motel,aguascalientes,aguascalientes,no se detecto incumplimiento,visita correcta


### Catálogo de estados

In [10]:
states_df = pd.DataFrame({'state_name':products_df['ENTIDAD'].unique()})
states_df['state_id'] = list(range(1, states_df.shape[0]+1))
states_df = states_df[['state_id', 'state_name']]
states_df.to_csv('data/states.csv', index=False)
print(f'Número de estados: {states_df.shape[0]}')
states_df.head()

Número de estados: 32


,state_id,state_name
0,1,aguascalientes
1,2,baja california
2,3,baja california sur
3,4,campeche
4,5,distrito federal


### Catálogo de tiendas

In [11]:
products_df.iloc[:, 7:].head()

,RAZON_SOCIAL,GIRO_COMERCIAL,MUNICIPIO,ENTIDAD,INCUMPLIMIENTO,RESULTADO
0,julio cepeda jugueterias s.a. de c.v.,jugueteria,aguascalientes,aguascalientes,no se detecto incumplimiento,visita correcta
1,c&a de mexico s. de r.l.,tienda departamental,aguascalientes,aguascalientes,no se detecto incumplimiento,visita correcta
2,asfeixas s.a. de c.v.,motel,aguascalientes,aguascalientes,no se detecto incumplimiento,visita correcta
3,motel villa miro- miro hoteleria s.a. de c.v.,motel,aguascalientes,aguascalientes,no se detecto incumplimiento,visita correcta
4,motel lua- compa�ia hotelera pescadores s.a. d...,motel,aguascalientes,aguascalientes,no se detecto incumplimiento,visita correcta


In [12]:
separator = '-*+*+-'
store_series = pd.unique(products_df['RAZON_SOCIAL'] + separator + products_df['GIRO_COMERCIAL'] + separator + products_df['MUNICIPIO'] + separator + products_df['ENTIDAD'])
stores_df = pd.DataFrame(
    {
        'store_unique_name': store_series
    }
)
print(f"Tiendas originales: {products_df.shape[0]}")
print(f"Tiendas únicas: {stores_df.shape[0]}")
stores_df.head()

Tiendas originales: 31214
Tiendas únicas: 12133


,store_unique_name
0,julio cepeda jugueterias s.a. de c.v.-*+*+-jug...
1,c&a de mexico s. de r.l.-*+*+-tienda departame...
2,asfeixas s.a. de c.v.-*+*+-motel-*+*+-aguascal...
3,motel villa miro- miro hoteleria s.a. de c.v.-...
4,motel lua- compa�ia hotelera pescadores s.a. d...


In [13]:
def get_state_id_from(state_name):
    return states_df[states_df['state_name'] == state_name]['state_id'].values[0]

In [14]:
get_state_id_from('distrito federal')

5

In [15]:
stores = []
purposes = []
counties = []
state_ids = []
for idx, row in stores_df.iterrows():
    try:
        e = row['store_unique_name'].split(separator)
        stores.append(e[0])
        purposes.append(e[1])
        counties.append(e[2])
        state_ids.append(get_state_id_from(e[3]))
    except Exception:
        print(e)
        print(row['store_unique_name'])
        break

In [16]:
print(np.array(stores[:10]))
print(np.array(purposes[:10]))
print(np.array(counties[:10]))
print(np.array(state_ids[:10]))

['julio cepeda jugueterias s.a. de c.v.' 'c&a de mexico s. de r.l.'
 'asfeixas s.a. de c.v.' 'motel villa miro- miro hoteleria s.a. de c.v.'
 'motel lua- compa�ia hotelera pescadores s.a. de c.v.'
 'motel pedra- pedra s.a. de c.v.'
 'candy street- claudia stephanie mora garcia'
 'vicky boutique- rocio esparza ortiz'
 'motel castillo-cia. hotelera fentos s. a. de c. v.'
 'motel los arcos s.a. de c.v.']
['jugueteria' 'tienda departamental' 'motel' 'motel' 'motel' 'motel'
 'zapateria' 'ropa para dama' 'hotel' 'bar']
['aguascalientes' 'aguascalientes' 'aguascalientes' 'aguascalientes'
 'aguascalientes' 'aguascalientes' 'aguascalientes' 'aguascalientes'
 'aguascalientes' 'jesus maria']
[1 1 1 1 1 1 1 1 1 1]


In [17]:
stores_df = pd.DataFrame(
    {
        'store_id':list(range(1, len(stores)+1)),
        'store_name':stores,
        'store_purpose':purposes,
        'store_county':counties,
        'state_id':state_ids
    }
)
stores_df.head()

,store_id,store_name,store_purpose,store_county,state_id
0,1,julio cepeda jugueterias s.a. de c.v.,jugueteria,aguascalientes,1
1,2,c&a de mexico s. de r.l.,tienda departamental,aguascalientes,1
2,3,asfeixas s.a. de c.v.,motel,aguascalientes,1
3,4,motel villa miro- miro hoteleria s.a. de c.v.,motel,aguascalientes,1
4,5,motel lua- compa�ia hotelera pescadores s.a. d...,motel,aguascalientes,1


In [18]:
stores_df.to_csv('data/stores.csv', index=False)

### Catálogo de productos

In [36]:
products_df.iloc[:, 5:7].head()

,TIPO_PRODUCTO,DESCRIPCION_PRODUCTO
0,mu�ecos,en empaque de carton estampado y plastico rigi...
1,ropa caballero,camisa para caballero en color azul marino de ...
2,brandy,una botella brandy de la marca torres 10 con l...
3,brandy,envase de vidrio contenido brandy de la marca ...
4,brandy,botella de vidrio contenido brandy 100% uva gr...


In [37]:
t = products_df.iloc[:, 5:]
t[(t["TIPO_PRODUCTO"] == 'brandy') & (t['RAZON_SOCIAL'] == 'cadena comercial oxxo s.a. de c.v.')].iloc[:, :6].head()

,TIPO_PRODUCTO,DESCRIPCION_PRODUCTO,RAZON_SOCIAL,GIRO_COMERCIAL,MUNICIPIO,ENTIDAD
3189,brandy,brandy penedes marca torres 5 en botella de vi...,cadena comercial oxxo s.a. de c.v.,tiendas de conveniencia,alvaro obregon,distrito federal
3197,brandy,brandy marca torres 5 en envase de vidrio cont...,cadena comercial oxxo s.a. de c.v.,tiendas de conveniencia,benito juarez,distrito federal
7218,brandy,brandy penedes 100% de uva en botella de vidri...,cadena comercial oxxo s.a. de c.v.,tiendas de conveniencia,cuauhtemoc,distrito federal
11957,brandy,botella de vidrio cont. brandy de 700ml. hecho...,cadena comercial oxxo s.a. de c.v.,tiendas de conveniencia,tlalnepantla de baz,mexico
12736,brandy,brandy penedes marca torres 5 100% uva solera ...,cadena comercial oxxo s.a. de c.v.,abarrotes vinos y licores,durango,durango


In [41]:
unique_products_series = pd.unique(products_df['TIPO_PRODUCTO'] + separator + products_df['DESCRIPCION_PRODUCTO'])
categories = []
products = []
for p in unique_products_series:
    e = p.split(separator)
    categories.append(e[0])
    products.append(e[1])

In [43]:
unique_products_df = pd.DataFrame(
    {
        'product_id': list(range(1, len(products)+1)),
        'product_name': products,
        'product_category': categories
    }
)
print(unique_products_df.shape)
unique_products_df.head()

(27720, 3)


,product_id,product_name,product_category
0,1,en empaque de carton estampado y plastico rigi...,mu�ecos
1,2,camisa para caballero en color azul marino de ...,ropa caballero
2,3,una botella brandy de la marca torres 10 con l...,brandy
3,4,envase de vidrio contenido brandy de la marca ...,brandy
4,5,botella de vidrio contenido brandy 100% uva gr...,brandy


In [44]:
unique_products_df.to_csv('data/products.csv', index=False)

### Catálogo Productos Tiendas

In [45]:
products_df.iloc[:, 5:].head()

,TIPO_PRODUCTO,DESCRIPCION_PRODUCTO,RAZON_SOCIAL,GIRO_COMERCIAL,MUNICIPIO,ENTIDAD,INCUMPLIMIENTO,RESULTADO
0,mu�ecos,en empaque de carton estampado y plastico rigi...,julio cepeda jugueterias s.a. de c.v.,jugueteria,aguascalientes,aguascalientes,no se detecto incumplimiento,visita correcta
1,ropa caballero,camisa para caballero en color azul marino de ...,c&a de mexico s. de r.l.,tienda departamental,aguascalientes,aguascalientes,no se detecto incumplimiento,visita correcta
2,brandy,una botella brandy de la marca torres 10 con l...,asfeixas s.a. de c.v.,motel,aguascalientes,aguascalientes,no se detecto incumplimiento,visita correcta
3,brandy,envase de vidrio contenido brandy de la marca ...,motel villa miro- miro hoteleria s.a. de c.v.,motel,aguascalientes,aguascalientes,no se detecto incumplimiento,visita correcta
4,brandy,botella de vidrio contenido brandy 100% uva gr...,motel lua- compa�ia hotelera pescadores s.a. d...,motel,aguascalientes,aguascalientes,no se detecto incumplimiento,visita correcta


In [46]:
stores_df.head()

,store_id,store_name,store_purpose,store_county,state_id
0,1,julio cepeda jugueterias s.a. de c.v.,jugueteria,aguascalientes,1
1,2,c&a de mexico s. de r.l.,tienda departamental,aguascalientes,1
2,3,asfeixas s.a. de c.v.,motel,aguascalientes,1
3,4,motel villa miro- miro hoteleria s.a. de c.v.,motel,aguascalientes,1
4,5,motel lua- compa�ia hotelera pescadores s.a. d...,motel,aguascalientes,1


In [47]:
unique_products_df.head()

,product_id,product_name,product_category
0,1,en empaque de carton estampado y plastico rigi...,mu�ecos
1,2,camisa para caballero en color azul marino de ...,ropa caballero
2,3,una botella brandy de la marca torres 10 con l...,brandy
3,4,envase de vidrio contenido brandy de la marca ...,brandy
4,5,botella de vidrio contenido brandy 100% uva gr...,brandy


In [48]:
def get_store_id_from(name, county, state):
    state_id = get_state_id_from(state)
    return stores_df[
        (stores_df['store_name'] == name) &
        (stores_df['store_county'] == county) &
        (stores_df['state_id'] == state_id)
    ]['store_id'].values[0]

def get_product_id_from(name, category):
    return unique_products_df[(unique_products_df['product_name'] == name) & (unique_products_df['product_category'] == category)]['product_id'].values[0]

In [49]:
get_store_id_from('cadena comercial oxxo s. a. de c.v.', 'aguascalientes', 'aguascalientes')

17

In [54]:
get_product_id_from('en empaque de carton estampado y plastico rigido mu�eca para ni�a de la marca barbie de origen china. contenido una pieza modelo barbie basica life fashion con numero de lote t7439 indica informacion importante para consulta impo', 'mu�ecos')

1

In [56]:
product_ids = []
store_ids = []
missing_reqs = []
for idx, row in products_df.iterrows():
    try:
        category = row['TIPO_PRODUCTO']
        product = row['DESCRIPCION_PRODUCTO']
        store_name = row['RAZON_SOCIAL']
        store_county = row['MUNICIPIO']
        store_state = row['ENTIDAD']
        product_ids.append(get_product_id_from(product, category))
        store_ids.append(get_store_id_from(store_name, store_county, store_state))
        missing_reqs.append(row['INCUMPLIMIENTO'])
    except Exception:
        print(row)
        break

In [57]:
product_store_df = pd.DataFrame(
    {
        'product_id': product_ids,
        'store_id': store_ids,
        'missing':missing_reqs
    }
)
print(product_store_df.shape)
product_store_df.head()

(31214, 3)


,product_id,store_id,missing
0,1,1,no se detecto incumplimiento
1,2,2,no se detecto incumplimiento
2,3,3,no se detecto incumplimiento
3,4,4,no se detecto incumplimiento
4,5,5,no se detecto incumplimiento


In [58]:
product_store_df.to_csv('data/product_store.csv', index=False)

## Registro de compras de usuario

### Catálogo de clientes

In [97]:
num_clients = 50000
clients_df = pd.DataFrame(
    {
        'client_id': list(range(1, num_clients+1)),
    }
)
print(clients_df.shape)
clients_df.head()

(50000, 1)


,client_id
0,1
1,2
2,3
3,4
4,5


In [98]:
clients_df.to_csv('data/clients.csv', index=False)

### Generación de órdenes de compra

In [99]:
max_purchases_per_client = 15
num_stores = stores_df.shape[0]

client_ids = []
store_ids = []
product_ids = []

for idx, row in clients_df.iterrows():
    client_id = row['client_id']
    num_purchases = np.random.randint(1, max_purchases_per_client+1)
    store_ids_nums = np.random.randint(1, num_stores+1, num_purchases)
    products_per_store = product_store_df[product_store_df['store_id'].isin(store_ids_nums)]['product_id']
    for sid in store_ids_nums:
        products_per_store = product_store_df[product_store_df['store_id'] == sid]['product_id']
        for prod in products_per_store:
            if np.random.randint(1, 200) < 100:
                times = np.random.randint(1, 4)
                for time in range(times):
                    client_ids.append(client_id)
                    store_ids.append(sid)
                    product_ids.append(prod)

In [100]:
purchases_df = pd.DataFrame(
    {
        'client_id':client_ids,
        'store_id':store_ids,
        'product_id':product_ids
    }
)
print(purchases_df.shape)
purchases_df.head()

(1022928, 3)


,client_id,store_id,product_id
0,1,1325,1672
1,1,1325,1672
2,1,169,166
3,1,169,166
4,1,9081,22418


In [101]:
purchases_df.to_csv('data/purchases.csv', index=False)